In [ ]:
import numpy as np 
from keras import backend as K 
from keras.datasets import mnist 
from keras.layers import (#Import the necessary layers here)
from keras.layers.advanced_activations import LeakyReLU 
from keras.optimizers import Adam 
from keras.models import Model,Sequential 
from keras.utils import to_categorical 

In [ ]:
#Model input dimensions 
imgRows = 28
imgCols = 28
channels = 1
imgShape = (imgRows,imgCols,channels)
zDim = 100
nClasses = 10

In [ ]:
class Dataset:
    def __init__(self,nLabeled):
        self.nLabeled = nLabeled 
        (self.x_train,self.y_train),(self.x_test,self.y_test) = mnist.load_data()
    
    def preProcessImgs(x):
        x = (x.astype(np.float32) - 127.5)/127.5
        x = np.expand_dims(x,axis = 3)
        return x 
    
    def preProcessLabels(y):
        return y.reshape(-1,1)
    
    self.x_train = preProcessImgs(self.x_train)
    self.x_test = preProcessImgs(self.x_test)
    self.y_train = preProcessLabels(self.y_train)
    self.y_test = preProcessLabels(self.y_test)
    
    def batchLabeled(self,batch_size):
        idx = np.random.randint(0,self.nLabeled,batch_size)
        imgs = self.x_train[idx]
        labels = self.y_train[idx]
        return imgs,labels
    
    def batchUnlabeled(self,batch_size):
        idx = np.random.randint(self.nLabeled,self.x_train.shape[0],batch_size)
        imgs = self.x_train[idx]
        return imgs 
    
    def training_set(self):
        x_train = self.x_train[range(self.nLabeled)]
        y_train = self.y_train[range(self.nLabeled)]
        return x_train,y_train 
    
    def test_set(self):
        return self.x_test,self.y_test
    
        

In [ ]:
def buildGenerator(zDim):
    model = Sequential()
    model.add(Dense(7*7*256,input_dim = zDim))
    model.add(Reshape((7,7,256)))
    model.add(Conv2DTranspose(128,kernel_size = 3,strides = 2,padding = 'same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha = 0.01))
    model.add(Conv2DTranspose(64,kernel_size = 3,strides = 1,padding = 'same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha = 0.01))
    model.add(Conv2DTranspose(1,kernel_size = 3,strides = 2,padding = 'same'))
    model.add(Activation('tanh'))
    return model 
    

In [ ]:
def buildDiscriminator_net(imgShape):
    model = Sequential()
    model.add(Conv2D(32,kernel_size = 3,strides = 2,padding = 'same',input_shape = imgShape))
    model.add(LeakyReLU(alpha = 0.01))
    model.add(Conv2D(64,kernel_size = 3,strides = 2,padding = 'same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha = 0.01))
    model.add(Conv2D(128,kernel_size = 3,strides = 2,padding = 'same'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(nClasses))
    return model 
    

In [ ]:
def discriminatorSupervised(D):
    #The discriminator to build the supervised part of the training 
    model = Sequential()
    model.add(D)
    model.add(Activation('softmax'))
    return model 

In [ ]:
def predict(x):
    prediction = 1.0 - (1.0/(K.sum(K.exp(x),axis = -1,keepdims = True)+1.0))
    return prediction

In [ ]:
def discriminatorUnsupervised(D):
    #Unsupervised part of the training 
    model = Sequential()
    model.add(D)
    model.add(Lambda(predict))
    return model 

In [ ]:
def buildGAN(G,D):
    model = Sequential()
    model.add(G)
    model.add(D)
    return model

In [ ]:
DNet = buildDiscriminator_net(imgShape)
DSupervised = discriminatorSupervised(DNet)
DSupervised.compile(loss = 'categorical_crossentropy',optimizer = Adam(),metrics = ['accuracy'])
DUnsupervised = discriminatorUnsupervised(DNet)
DUnsupervised.compile(loss = 'binary_crossentropy',optimizer = Adam())
Generator = buildGenerator(zDim)
DUnsupervised.trainable = False 
SGAN = buildGAN(Generator,DUnsupervised)
SGAN.compile(loss = 'binary_crossentropy',optimizer = Adam())

In [ ]:
supervised_losses = []; accuracies = []
nLabeled = 100 #Mini-batch size 
#Get the labeled and unlabeled images
dataset = Dataset(nLabeled)
def trainGAN(iterations,batch_size,sampleInterval):
    real = np.ones((batch_size,1))
    fake = np.zeros((batch_size,1))
    for iteration in range(iterations):
        #Get a batch of Both labeled and unlabeled images
        imgs,labels = dataset.batchLabeled(batch_size)
        labels = to_categorical(labels,nClasses)
        imgsUnlabeled = dataset.batchUnlabeled(batch_size)
        
        #Train the Supervised discriminator 
        z = np.random.normal(0,1,(batch_size,zDim))
        genImgs = generator.predict(z)
        dLossSupervised,accuracy = DSupervised.train_on_batch(imgs,labels)
        
        #Train the unsupervised discriminator 
        dLossReal = DUnsupervised.train_on_batch(imgsUnlabeled,real)
        dLossFake = DUnsupervised.train_on_batch(genImgs,fake)
        dLossUnsupervised = 0.5*np.add(dLossReal,dLossFake)
        
        #Train the generator
        z = np.random.normal(0,1,(batch_size,zDim))
        genImgs = generator.predict(z)
        gLoss = generator.train_on_batch(z,np.ones((batch_size,1)))
        
        if(iteration+1)%sampleInterval == 0:
            supervised_losses.append(dLossSupervised)
            accuracies.append(accuracy)
            
    